In [1]:
!pip install transformers==3.0.2

     |████████████████████████████████| 778kB 4.5MB/s 
     |████████████████████████████████| 1.1MB 18.6MB/s 
     |████████████████████████████████| 890kB 30.0MB/s 
     |████████████████████████████████| 3.0MB 42.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=6d959398e214372aa4aefaa751ec60d4fd5e3edfd7cd7532de062bdc7968d107
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import numpy as np
import pandas as pd
import nltk
import sys
nltk.download('punkt')
import transformers
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=transformers.BertModel.from_pretrained('bert-base-uncased').to(device).eval()
tokenizer=transformers.BertTokenizer.from_pretrained('bert-base-uncased')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
def get_tfidf_features(paragraphs):
    print("Extracting tf-idf features")
    all_paragraph_features=[]
    vectorizer = TfidfVectorizer()
    lengths=[]
    for paragraph in paragraphs:
        vectorizer = TfidfVectorizer()
        curr_features=vectorizer.fit_transform(paragraph)
        lengths.append(len(vectorizer.get_feature_names()))
        all_paragraph_features.append(np.array(curr_features.todense()))
    lengths=np.array(lengths)
    print("Min length of dictionary: {}".format(np.amin(lengths)))
    print("Avg length of dictionary: {}".format(np.mean(lengths)))
    print("Max length of dictionary: {}".format(np.amax(lengths)))
    return np.asarray(all_paragraph_features)

In [10]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-11-09 15:26:46--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-11-09 15:26:46--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-11-09 15:26:46--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [11]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [12]:
!python read_glove.py

tcmalloc: large alloc 1105485824 bytes == 0xceef2000 @  0x7f7394ce12a4 0x591e47 0x4dd737 0x4e2b00 0x4e214a 0x4e0df8 0x4e255b 0x4e29d0 0x4e33c6 0x4544bc 0x50a25a 0x50beb4 0x507be4 0x50ad03 0x634e72 0x634f27 0x6386df 0x639281 0x4b0dc0 0x7f73948ddb97 0x5b259a


In [14]:
#READING GLOVE FILE
import pickle
word_to_index={}
index_to_word={}
word_to_vec_map={}
fp= open('word_to_index.pkl', 'rb')
word_to_index = pickle.load(fp)
fp= open('index_to_word.pkl', 'rb')
index_to_word = pickle.load(fp)
fp= open('word_to_vec_map.pkl', 'rb')
word_to_vec_map = pickle.load(fp)
fp.close()

In [16]:
word_to_vec_map['hello'].shape

(300,)

In [28]:
embed_dimension=300
def get_word_embedding_features(paragraphs):
    word_embed_features=[]
    for i,paragraph in enumerate(paragraphs):
        sys.stdout.write("\rExtracted features for {} of {}".format(i+1,paragraphs.shape[0]))
        sys.stdout.flush()
        curr_embed_features=[]
        num_sentences=len(paragraph)
        for sentence in paragraph:
            sentence_lower=sentence.lower()
            sentence_words=nltk.TreebankWordTokenizer().tokenize(sentence_lower)
            curr_vec=np.zeros((embed_dimension))
            for w in sentence_words:
                if w in word_to_vec_map:
                    curr_vec+=word_to_vec_map[w]
            curr_vec/=num_sentences
            curr_embed_features.append(curr_vec)
        word_embed_features.append(np.asarray(curr_embed_features))
    return np.asarray(word_embed_features)

In [4]:
print("GPU/CPU:",torch.cuda.get_device_name(0)) 
def bert_features(paragraph):
    max_seq_length=500
    batch_tokens=[]
    longest_seq=0
    for sentence in paragraph:
        tokens=tokenizer.tokenize(sentence)
        tokens_id=tokenizer.convert_tokens_to_ids(tokens)
        longest_seq = max(longest_seq, len(tokens))
        batch_tokens.append(tokens_id)
    features = {}
    pad_seq_length = min(longest_seq,max_seq_length) + 3
    for text in batch_tokens:
        sentence_features = tokenizer.prepare_for_model(text,max_length=pad_seq_length, pad_to_max_length=True, return_tensors='pt',truncation=True)
        for feature_name in sentence_features:
            if feature_name not in features:
                features[feature_name] = []
            features[feature_name].append(sentence_features[feature_name])
    for feature_name in features:
        features[feature_name] = torch.cat(features[feature_name]).to(device)
    with torch.no_grad():
        outputs = model(**features)
    pooled_output = outputs[1]
    return pooled_output
    
def get_bert_features(paragraphs):
    all_paragraph_features=[]
    for i,paragraph in enumerate(paragraphs):
        curr_features=bert_features(paragraph)
        sys.stdout.write("\rExtracted features for {} of {}".format(i+1,paragraphs.shape[0]))
        sys.stdout.flush()
        all_paragraph_features.append(curr_features)
    return np.asarray(all_paragraph_features)

GPU/CPU: Tesla T4


In [5]:
from nltk.cluster.util import cosine_distance
def build_similarity_matrix(paragraph_features,is_tensor):
    if is_tensor:
        paragraph_features=paragraph_features.cpu().numpy()
    num_sentences=paragraph_features.shape[0]
    similarity_matrix = np.zeros((num_sentences, num_sentences))
    for idx1 in range(num_sentences):
        for idx2 in range(num_sentences):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = 1-cosine_distance(paragraph_features[idx1],paragraph_features[idx2])
    return similarity_matrix

import networkx as nx
def generate_summaries(paragraphs,features_paragraphs,is_tensor=False):
    text_summaries=[]
    cnt=0
    for i,paragraph_features in enumerate(features_paragraphs):
        sys.stdout.write("\rGenerating summary for {} of {}".format(i+1,paragraphs.shape[0]))
        sys.stdout.flush()
        similarity_matrix=build_similarity_matrix(paragraph_features,is_tensor)
        sentence_similarity_graph = nx.from_numpy_array(similarity_matrix)
        try:
            scores = nx.pagerank(sentence_similarity_graph,max_iter=100)
            ranked_sentence = sorted(((scores[j],j) for j,s in enumerate(paragraphs[i])), reverse=True)    
            top_n=3
            summary_indices=[]
            for j in range(top_n):
                summary_indices.append(ranked_sentence[j][1])
            out=np.array(paragraphs[i])
            out_indices=sorted(summary_indices)
            text_summaries.append(("".join(out),"".join(out[out_indices]),similarity_matrix))
        except:
            cnt+=1
    print("Pagerank didn't converge for {} paragraphs".format(cnt))
    return text_summaries

In [7]:
#DATASET-1
#Loading and cleaning
data=pd.read_csv('news_summary.csv',encoding='iso-8859-1')
data_paragraphs=data['ctext']
all_paragraphs=[]
cnt=0
for paragraph in data_paragraphs:
    try:
        all_paragraphs.append(nltk.sent_tokenize(paragraph))
    except:
        cnt+=1
print("{} paragraphs omitted".format(cnt))
all_paragraphs=np.array(all_paragraphs)
print("{} paragraphs parsed".format(all_paragraphs.shape[0]))
lengths=[]
min_thres=3
max_thres=35
paragraphs=[]
for paragraph in all_paragraphs:
    curr_len=len(paragraph)
    if curr_len>=min_thres and curr_len<=max_thres:
        paragraphs.append(paragraph)
print("{} paragraphs in dataset after applying length constraint".format(len(paragraphs)))
paragraphs=np.array(paragraphs)

118 paragraphs omitted
4396 paragraphs parsed
3878 paragraphs in dataset after applying length constraint


In [9]:
#Visualizing
for paragraph in paragraphs:
    if len(paragraph)==min_thres:
        print(paragraph)
        break
print("\n")
for paragraph in paragraphs:
    if len(paragraph)==max_thres:
        print(paragraph)
        break

['President Donald Trump?s son-in-law says the Trump campaign couldn?t have colluded with Russia because the team was too dysfunctional and disorganised to coordinate with a foreign government.Jared Kushner, a senior adviser to the president, made the comment Monday during a closed-door session with congressional interns.A Democratic congressional aide says Kushner was responding to a question about Special Counsel Robert Mueller?s investigation into whether the Trump campaign colluded with Moscow.ForeignPolicy.com first reported Kushner?s remarks.', 'The aide was knowledgeable of the meeting and confirmed the accuracy of the comments.Last week, Kushner met privately at the Capitol with members of the Senate and House intelligence committees.He acknowledged four meetings with Russians during and after Trump?s victorious White House bid and insisted that he had ?nothing to hide.', '?']


['It took him five years and lot of courage to openly speak about his battle with drug addiction.', 

In [18]:
#TF-IDF FEATURES AND SUMMARIES

In [19]:
features_tfidf=get_tfidf_features(paragraphs)

Extracting tf-idf features
Min length of dictionary: 34
Avg length of dictionary: 170.2885507993811
Max length of dictionary: 538


In [20]:
tfidf_summaries=generate_summaries(paragraphs,features_tfidf)

Generating summary for 20 of 3878

/usr/local/lib/python3.6/dist-packages/nltk/cluster/util.py:133: RuntimeWarning: invalid value encountered in double_scalars
  sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))


Generating summary for 3878 of 3878Pagerank didn't converge for 218 paragraphs


In [ ]:
###GloVe embedding FEATURES AND SUMMARIES

In [29]:
features_glove=get_word_embedding_features(paragraphs)

Extracted features for 3878 of 3878

In [35]:
glove_summaries=generate_summaries(paragraphs,features_glove)

Generating summary for 932 of 3878

/usr/local/lib/python3.6/dist-packages/nltk/cluster/util.py:133: RuntimeWarning: invalid value encountered in double_scalars
  sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))


Generating summary for 3878 of 3878Pagerank didn't converge for 11 paragraphs


In [21]:
###BERT FEATURES AND SUMMARIES

In [22]:
features_bert=get_bert_features(paragraphs)

Extracted features for 3878 of 3878

In [23]:
bert_summaries=generate_summaries(paragraphs,features_bert,is_tensor=True)

Generating summary for 3878 of 3878Pagerank didn't converge for 37 paragraphs


In [ ]:
###ANALYSIS

In [24]:
bert_summaries[27][:2]

('The remains of a German hiker who disappeared while climbing in the Swiss Alps 30 years ago has been found embedded in a glacier, police said on Wednesday.The find was made on July 25 by two people climbing the Lagginhorn mountain, in southern Switzerland, police said in a statement.A few hundred metres before reaching the peak, the hikers spotted ?a hand and two shoes?, area police said.Extracting the remains that day was impossible because of poor weather, but rescue workers arrived by helicopter the following day, removed the remains from the ice and brought them to a hospital in the capital Bern for identification.Test have confirmed the climber was a German national born in 1943 who disappeared on August 11, 1987.The discovery came just two days after a Swiss couple who disappeared while walking in the Alps in 1942 were found in a receding glacier.',
 'The find was made on July 25 by two people climbing the Lagginhorn mountain, in southern Switzerland, police said in a statement

In [25]:
tfidf_summaries[25][:2]

('The remains of a German hiker who disappeared while climbing in the Swiss Alps 30 years ago has been found embedded in a glacier, police said on Wednesday.The find was made on July 25 by two people climbing the Lagginhorn mountain, in southern Switzerland, police said in a statement.A few hundred metres before reaching the peak, the hikers spotted ?a hand and two shoes?, area police said.Extracting the remains that day was impossible because of poor weather, but rescue workers arrived by helicopter the following day, removed the remains from the ice and brought them to a hospital in the capital Bern for identification.Test have confirmed the climber was a German national born in 1943 who disappeared on August 11, 1987.The discovery came just two days after a Swiss couple who disappeared while walking in the Alps in 1942 were found in a receding glacier.',
 'The remains of a German hiker who disappeared while climbing in the Swiss Alps 30 years ago has been found embedded in a glacier

In [39]:
glove_summaries[27][:2]

('The remains of a German hiker who disappeared while climbing in the Swiss Alps 30 years ago has been found embedded in a glacier, police said on Wednesday.The find was made on July 25 by two people climbing the Lagginhorn mountain, in southern Switzerland, police said in a statement.A few hundred metres before reaching the peak, the hikers spotted ?a hand and two shoes?, area police said.Extracting the remains that day was impossible because of poor weather, but rescue workers arrived by helicopter the following day, removed the remains from the ice and brought them to a hospital in the capital Bern for identification.Test have confirmed the climber was a German national born in 1943 who disappeared on August 11, 1987.The discovery came just two days after a Swiss couple who disappeared while walking in the Alps in 1942 were found in a receding glacier.',
 'The remains of a German hiker who disappeared while climbing in the Swiss Alps 30 years ago has been found embedded in a glacier